<a href="https://colab.research.google.com/github/PadmajaVB/Sexist-Statement-Detection/blob/main/DataExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Script to get the data for everyday Sexism project

In [2]:
!pip install ipdb

     |████████████████████████████████| 788kB 9.3MB/s 
     |████████████████████████████████| 368kB 37.1MB/s 
  Created wheel for ipdb: filename=ipdb-0.13.8-cp37-none-any.whl size=11599 sha256=7deeecdad7ec374911d0d8dfbabb814be13bf4264263f64591c7eb6c69b6b5e3
  Stored in directory: /root/.cache/pip/wheels/0d/d6/5a/2fdf30b75ca5099e18f66a0a4d439ba031e1aa239e12b39c24
Successfully built ipdb
ERROR: jupyter-console 5.2.0 has requirement prompt-toolkit<2.0.0,>=1.0.0, but you'll have prompt-toolkit 3.0.18 which is incompatible.
ERROR: google-colab 1.0.0 has requirement ipython~=5.5.0, but you'll have ipython 7.24.0 which is incompatible.
  Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Found existing installation: ipython 5.5.0
    Uninstalling ipython-5.5.0:
      Successfully uninstalled ipython-5.5.0


In [3]:
!apt-get install libmagic-dev
!pip install python-magic

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libmagic-mgc libmagic1
Suggested packages:
  file
The following NEW packages will be installed:
  libmagic-dev libmagic-mgc libmagic1
0 upgraded, 3 newly installed, 0 to remove and 34 not upgraded.
Need to get 332 kB of archives.
After this operation, 5,552 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-mgc amd64 1:5.32-2ubuntu0.4 [184 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic1 amd64 1:5.32-2ubuntu0.4 [68.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-dev amd64 1:5.32-2ubuntu0.4 [79.7 kB]
Fetched 332 kB in 1s (289 kB/s)
Selecting previously unselected package libmagic-m

In [4]:
!pip install ftfy

     |████████████████████████████████| 71kB 5.3MB/s 
  Created wheel for ftfy: filename=ftfy-6.0.3-cp37-none-any.whl size=41916 sha256=5b7f41ad8ddcfa3f3617c8301dea126ca0895145cfc2fef78f570f446c3bdf19
  Stored in directory: /root/.cache/pip/wheels/99/2c/e6/109c8a28fef7a443f67ba58df21fe1d0067ac3322e75e6b0b7
Successfully built ftfy


In [5]:
import re
import urllib.request as urllib2
import os
import ipdb
from bs4 import BeautifulSoup
import time
import sys
import codecs
import pandas as pd
import csv
import json
from os import listdir
from os.path import isfile, join
import io
import magic
import pickle
import ftfy

In [ ]:
tsv_filepath = '/content/drive/MyDrive/UnivAI/AI-3 Project/data/Everyday Sexism Project/data_placeholders.tsv'
data_write_path = '/content/drive/MyDrive/UnivAI/AI-3 Project/data/Everyday Sexism Project/data.tsv'


def find_post_id(name_box):
    classes = name_box['class']
    for _class in classes:
        if _class[:5] == "post-" and len(_class) > 5:
            return int(_class[5:])
    return -1


def crawl_post(post_page, post_number):
    crawl_page = post_page + str(post_number)

    isCrawledSuccessfully = False
    while not isCrawledSuccessfully:
        try:
            page = urllib2.urlopen(crawl_page).read()
            isCrawledSuccessfully = True
        except urllib2.HTTPError as e:
            if e.code == 404:
                print("\t\t>>> Found 404")
                return None
            else:
                print("\t\t>>> Waitiing for 30 seconds")
                time.sleep(30)
        except Exception:
            print("\t\t>>> Waitiing for 30 seconds")
            time.sleep(30)

    soup = BeautifulSoup(page, 'html.parser')
    name_boxes = soup.find_all('article', attrs={'class': 'post'})
    assert len(name_boxes) == 1
    for i, name_box in enumerate(name_boxes):
        post_id = find_post_id(name_box)
        if post_id == -1:
            print("\t\t>>> ERROR: Error fetching post_id")
            return None
        
        #TODO: VERIFY IF THIS IS CORRECT
        soup = BeautifulSoup(name_box.encode("utf-8"), 'html.parser')
        _name_box = soup.find('div', attrs={'class': 'entry-content'})
        post = _name_box.get_text()
        post = post.replace('\n', '<br/>')
    return post


import re

r_white = re.compile(r'\s+')

xml_csv = open(data_write_path, 'w')
csv_writer = csv.writer(xml_csv, delimiter='\t')
csv_writer.writerow(['post', 'labels'])

post_id_dict = {}

with open(tsv_filepath, 'r') as csvfile:
    spamreader = csv.reader(csvfile, delimiter='\t')
    for i, row in enumerate(spamreader):
        if i == 0:
            continue
        labels = row[1]
        parts = row[0].split("__")
        post_id__str = parts[0]
        post_offsets__str = [offset.split("_") for offset in parts[1:]]

        if post_id__str in post_id_dict:
            print("accessing " + post_id__str)
            post_text = post_id_dict[post_id__str]
        else:
            print("fetching " + post_id__str)
            post_text = crawl_post("https://everydaysexism.com/everyday-sexism/", post_id__str)
            if post_text == None:
                continue
            post_id_dict[post_id__str] = post_text

        post_text = post_text.replace('<br/>', ' ')
        post_text = re.sub(' +', ' ', post_text)
        post_text = post_text.lstrip()
        post_text__fixed = ftfy.fix_text(post_text)
        text = ""
        for index in range(len(post_offsets__str)):
            text = text + post_text__fixed[int(post_offsets__str[index][0]):int(post_offsets__str[index][1]) + 1]
            if len(post_offsets__str) > 1 and len(post_offsets__str) != index + 1:
                text = text + " "
        text = r_white.sub(' ', text)
        text = text.strip()
        csv_writer.writerow([text, labels])

xml_csv.close()

In [ ]:
df_es = pd.read_csv(data_write_path, sep='\t')

In [2]:
path = '/content/drive/MyDrive/UnivAI/AI-3 Project/data/Everyday Sexism Project/data.tsv'
data = pd.read_csv(path, sep='\t')

In [3]:
data.head()

,post,labels
0,"My cousin (female, and 14 years older than me)...","Attribute stereotyping, Body shaming, Denial o..."
1,Big brother came to hang out with us toward th...,Sexual assault (excluding rape)
2,"The next morning, I told my friend. She didn't...","Denial or trivialization of sexist misconduct,..."
3,My ex-boyfriend tried guilt tripping me into s...,"Sexual harassment (excluding assault), Slut sh..."
4,While I was working at McDonald's there was th...,"Hostile work environment (excluding pay gap), ..."


In [4]:
'Moral policing' in data['labels'][0]

True

In [5]:
label_list = ['Attribute stereotyping', 'Role stereotyping', 'Pay gap', 'Hostile work environment', 'Tone policing', 'Moral policing', 'Mansplaining']

In [6]:
def transform(text, label_list):
  for label in label_list:
    if label in text:
      return 1
  return 0

In [7]:
data['labels'] = data['labels'].apply(transform,label_list=label_list)

In [8]:
data.head()

,post,labels
0,"My cousin (female, and 14 years older than me)...",1
1,Big brother came to hang out with us toward th...,0
2,"The next morning, I told my friend. She didn't...",0
3,My ex-boyfriend tried guilt tripping me into s...,0
4,While I was working at McDonald's there was th...,1


In [9]:
data = data[data['labels']==1]

In [10]:
data.head()

,post,labels
0,"My cousin (female, and 14 years older than me)...",1
4,While I was working at McDonald's there was th...,1
5,I told multiple supervisors and the head of th...,1
6,I worked as a Project Engineer at a major comp...,1
9,"I wanted to go in the forces and was told ""Wom...",1


In [11]:
data.rename(columns = {'post':'Sentences', 'labels':'Label'}, inplace=True)

In [12]:
len(data)

6326

In [13]:
data.head()

,Sentences,Label
0,"My cousin (female, and 14 years older than me)...",1
4,While I was working at McDonald's there was th...,1
5,I told multiple supervisors and the head of th...,1
6,I worked as a Project Engineer at a major comp...,1
9,"I wanted to go in the forces and was told ""Wom...",1


In [14]:
len_list = [len(x) for x in data.Sentences.values]

In [15]:
max(len_list)

879

In [16]:
min(len_list)

14

In [19]:
temp = pd.DataFrame(len_list)

In [22]:
len(temp[temp[0]>500])

45

In [25]:
 max_len = data['Sentences'].apply(len).max()

In [26]:
max_len

879

In [27]:
data.to_csv('/content/drive/MyDrive/UnivAI/AI-3 Project/data/Preprocessed data/everyday_sexist_comment.csv')